In [2]:
# 처음 실행되는 img_draw창에서 중앙에 있는 빨간 사격형 부분이 색을 읽는 부분임
# 해당 부분에 영상에 그리고 싶은 색의 물체를 올려놓고 space bar를 누름
# 왼쪽 img_result 창의 트랙바를 이용하여 확실하게 색을 잡아줌
# 이후 r 버튼을 누르면 원하는 색의 물체의 중심에 보이는 원을 기준으로 선이 그려짐
# 다시 r 버튼을 누르면 그리는 행위를 일시 중지함
# 이후 다시 space bar를 누르면 색을 얻기위한 빨간 사각형부분이 뜨며 다시 space bar를 누르면 원하는 색상을 바꿔줄 수 있음
# x버튼을 누르면 화면이 전부 지워짐
#
# m키를 누르면 스티커를 만드는 화면이 뜸
# simg 창에서 원하는 색상을 찾아 마우스 왼쪽 클릭을 하면 해당 색이 지정이 됨
# 이후 마우스를 왼쪽클릭으로 드래그 앤 드롭 하면 해당 색으로 그림이 그려짐
# 새로운색을 얻고 싶으면 n 버튼을 눌러 원하는 색의 부분을 클릭하면 됨
# e를 누르면 검은색으로 덫칠하여 지우개의 역할을 수행함 이후 다시 e를 누르면 지우개 모드가 종료되고 다시 색을 지정하여 그림을 그리면 됨
# 만약 스티커를 초기화 하고 싶으면 c를 누름
# 이후 스티커를 다 만들었으면 esc 버튼을 누른 후, img_draw 창을 클릭해준 뒤 s를 누르면 스티커가 화면에 그려짐.

# 스티커를 만들때 hsv 이미지를 가져와 색상을 선택하기때문에 다양한 색상을 가진 이미지가 필요.
#simg = cv2.imread("./img/1.jpg") 이부분 변경 요망

import cv2
import numpy as np

###변수 설정들
cap = cv2.VideoCapture(0)

color1 = 0 #이미지에서 읽어온 h색상의 평균값을 저장하는곳
step = 0 # 색을 바꾸기위해 임시로 설정한 변수
ranges = 20 # h값 조절 범위
isDraw = False
set_color = False # 동영상 위에 그릴 색이 세팅이 되었는가  t f
lower_color1=0
lower_color2=0
lower_color3=0
upper_color1=0
upper_color2=0
upper_color3=0
is_stikerdraw=False

list_draw_location = [] # 현재 좌표들과 해당 색상을 저장하는 배열
history_draw_locations = [] # draw가 f로 바뀌면 현재 좌표들과 색을 받아와 저장(화면에 계속 띄어주기위함)

set_sColor = False # 스티커에서 그릴 색이 세팅이 되었는가 t f
is_sDragging = False # 스티커를 만들때 현재 좌표로 그릴지 말지를 결정 t f
is_ser=False # 스티커에서 지우개를 사용할건가 말것인가 t f
scolor=0 # 스티커에 그림을 그릴 색을 저장하는 변수
is_stiker=False # 스티커를 만들것인지 말것인지를 결정 t f
is_result=False # 스티커를 다 만든후 원본 동영상에 붙일것인지 말것인지.

list_sColor=[] # 스티커에 그릴 색상들을 저장할 변수
list_sDraw=[] #  스티커에서 현재 좌표를 이용해 그림을 그릴때 쓸 좌표 저장
history_list_sDraw=[] # 스티커에서 is_sDragging가 f로 바뀌면 현재좌표들을 이곳에 추가로 저장

def mouse_callback2(event, x, y, flags, param):
    global point1,point2,is_stikerdraw
    if event==cv2.EVENT_LBUTTONDOWN:
        point1=x
        point2=y
        is_stikerdraw=True
## 원본 동영상에 그림을 그리는 함수

def draw(img_draw, locations): 
    for i in range(len(locations)-1):
        #좌표와 색을 읽어서 선을 그림
        cv2.line(img_draw, tuple(locations[i][:2]), tuple(locations[i+1][:2]),locations[i][2], 3)

    return img_draw

## 원본 동영상에서 원하는 색상을 찾아내서 mask형식으로 만드는 함수

def makemask(x):
    global color1
    global lower_color1, lower_color2, lower_color3,upper_color1, upper_color2, upper_color3

    #hsv 색상에서 s v값을 트랙바로 변경하기 위함
    saturation = cv2.getTrackbarPos('saturation', 'img_result')
    value = cv2.getTrackbarPos('value', 'img_result')

    
    color1 = int(color1)

    # h의 범위로 +-20값을 얻어야하는데 각각 계산결과가 0보다작거나 180이 클때를 대비하여 
    # 세가지 구간으로 나누어 계산
    # 이때에 s v값은 트랙바값으로 받아옴
    
    if color1 < ranges:
        lower_color1 = np.array([color1 - ranges + 180, saturation, value])
        upper_color1 = np.array([180, 255, 255])
        lower_color2 = np.array([0, saturation, value])
        upper_color2 = np.array([color1, 255, 255])
        lower_color3 = np.array([color1, saturation, value])
        upper_color3 = np.array([color1 + ranges, 255, 255])
    
    elif color1 > 180 - ranges:
        lower_color1 = np.array([color1, saturation, value])
        upper_color1 = np.array([180, 255, 255])
        lower_color2 = np.array([0, saturation, value])
        upper_color2 = np.array([color1 + ranges - 180, 255, 255])
        lower_color3 = np.array([color1 - ranges, saturation, value])
        upper_color3 = np.array([color1, 255, 255])

    else:
        lower_color1 = np.array([color1, saturation, value])
        upper_color1 = np.array([color1 + ranges, 255, 255])
        lower_color2 = np.array([color1 - ranges, saturation, value])
        upper_color2 = np.array([color1, 255, 255])
        lower_color3 = np.array([color1 - ranges, saturation, value])
        upper_color3 = np.array([color1, 255, 255])

cv2.namedWindow('img_draw')
cv2.namedWindow('img_result')

# 트랙바 보여줄 window창의 이름, 범위와 할당할 변수 세팅

cv2.createTrackbar('saturation', 'img_result', 0, 255, makemask)
cv2.setTrackbarPos('saturation', 'img_result', 120)
cv2.createTrackbar('value', 'img_result', 0, 255, makemask)
cv2.setTrackbarPos('value', 'img_result', 120)

## 스티커를 만드는 함수

def makestiker():
    
    #스티커를 만들때 그림을 그리는 함수(위는 line으로 이건 원으로)
    def draw(img, locations):
        th=6
        for i in range(len(locations)-1):
            if locations[i][2]==(0,0,0): # 지우개의 설정값이 000 즉 검은색(마스크를 위함)
                th=20
            cv2.circle(img, ((locations[i][0]),(locations[i][1])),th,locations[i][2], -1)

        return img
    
    #스티커 만드는함수 내에서의 마우스 콜백함수
    def mouse_callback(event, x, y, flags, param):
        global set_sColor, is_sDragging,list_sColor,list_sDraw,history_list_sDraw,is_ser,scolor

        #스티커의 색상이 결정안된 상태에서
        if set_sColor == False:
            #지우개가 선택되면
            if is_ser == True:
                scolor=(0,0,0)
                set_sColor =True
            # 지우개가 아니며, 왼쪽버튼이 눌리면 색상을 선택
            elif is_ser == False and event==cv2.EVENT_LBUTTONDOWN:
                scolor = simg[y, x]
                scolor=(int(scolor[0]),int(scolor[1]),int(scolor[2]))
                list_sColor.append(scolor)
                set_sColor =True
        # 색상이 선택된 상태에서        
        elif set_sColor ==True:
            #왼쪽버튼을 누르면 그리기시작
            if event==cv2.EVENT_LBUTTONDOWN:
                is_sDragging = True
            # 움직이는동안은 좌표를 list에 저장
            elif event==cv2.EVENT_MOUSEMOVE:
                if is_sDragging:
                    list_sDraw.append((x,y,scolor))
            # 왼쪽버튼을 때면 그리기름 멈추고 좌표들을 history에 넘겨줌
            elif event==cv2.EVENT_LBUTTONUP:
                if is_sDragging:
                    is_sDragging=False
                    history_list_sDraw.append(list_sDraw.copy())
                    list_sDraw.clear()

    #스티커를 만드는 함수의 main
    while True:
        global is_ser,set_sColor
        # 그림을 받아온뒤 그 위에 내가 그린 그림들을 그림
        simg = cv2.imread("./img/1.jpg")
        simg_c=simg.copy()
        h4, w4 = simg.shape[:2]
        simg = draw(simg, list_sDraw)
        
        zoom = (cv2.getTrackbarPos('zoom', 'simgs'))/10
        degree = cv2.getTrackbarPos('degree', 'simgs')

        for sdraw_locations in history_list_sDraw:
            simg = draw(simg, sdraw_locations)
        # 스티커라 함은 내가 그린부분만 따서 원본 이미지에 붙이기 위함이다
        # 스티커 원본 이미지와 내가 그린 부분의 차이를 통해 스티커를 가져온다,
        # 해당작업을 위해선 이미지에서 내가 그린 부분만 따온후 배경은 검은색으로 바꾸어야한다
        
        diff=cv2.absdiff(simg,simg_c)
        diff1=cv2.cvtColor(diff,cv2.COLOR_BGR2GRAY)
        _,mask=cv2.threshold(diff1,1,255,cv2.THRESH_BINARY)
        mask1=cv2.bitwise_and(simg,simg,mask=mask)
        h3,w3=mask1.shape[:2]
        md=cv2.getRotationMatrix2D((w3/2,h3/2),degree,zoom)
        mask1=cv2.warpAffine(mask1,md,(w3,h3))
        mask1=cv2.resize(mask1,None,None,zoom,zoom)
        
        
        
        cv2.imshow('simg',simg)
        cv2.imshow('simgs',mask1)

        cv2.setMouseCallback('simg',mouse_callback)

        # 키세팅  esc종료 n 은 새로운 색 추출, c는 화면지우기 , e는 지우개
        skey=cv2.waitKey(1)
        if skey == 27: # esc
            return mask1
        elif skey == ord('n'):
            set_sColor=False
        elif skey == ord('c'):
            list_sDraw.clear()
            history_list_sDraw.clear()
        elif skey == ord('e'):
            is_ser=not is_ser
            set_sColor=not set_sColor
cv2.namedWindow('simgs',0)
cv2.createTrackbar('zoom', 'simgs', 2, 10, makestiker)
cv2.setTrackbarPos('zoom', 'simgs', 10)
cv2.createTrackbar('degree', 'simgs', 0, 360, makestiker)
cv2.setTrackbarPos('degree', 'simgs', 0)
    

### 본격적인 화면내 그림그리기 시작
while True:
    global point1,point2,is_stikerdraw
    ret,img_draw = cap.read()
    img_draw = cv2.flip(img_draw,1)
    cv2.setMouseCallback('img_draw',mouse_callback2)
    img_draw2 = img_draw.copy()
    img_hsv = cv2.cvtColor(img_draw2, cv2.COLOR_BGR2HSV)

    height, width = img_draw.shape[:2]
    point=[]
    cx = int(width / 2)
    cy = int(height / 2)
    
    # 원본 동영상에서 원하는 색을 찾았는가?
    if set_color == False:
        
        #색을 찾지 않았으면 화면의 중앙에 빨간색 사각형범위를 보여줌 (이곳을 이용하여 색을 얻어냄)
        cv2.rectangle(img_draw, (cx - 20, cy - 20), (cx + 20, cy + 20), (0, 0,255), 3)
    
    else:

        # 위에서 만든 마스크 3개를 or시킨후 원하는 색의 마스크를 얻음
        img_maskA1 = cv2.inRange(img_hsv, lower_color1, upper_color1)
        img_maskA2 = cv2.inRange(img_hsv, lower_color2, upper_color2)
        img_maskA3 = cv2.inRange(img_hsv, lower_color3, upper_color3)
        temp = cv2.bitwise_or(img_maskA1, img_maskA2)
        img_maskA = cv2.bitwise_or(img_maskA3, temp)
        
        # morphology를 이용하여 물체내의 검은점, 배경에서의 흰점들을 없애줌
        kernel = np.ones((11,11), np.uint8)
        img_maskA = cv2.morphologyEx(img_maskA, cv2.MORPH_OPEN, kernel)
        img_maskA = cv2.morphologyEx(img_maskA, cv2.MORPH_CLOSE, kernel)
        img_maskA = cv2.morphologyEx(img_maskA, cv2.MORPH_OPEN, kernel)
        img_maskA = cv2.morphologyEx(img_maskA, cv2.MORPH_CLOSE, kernel)
        img_maskA = cv2.morphologyEx(img_maskA, cv2.MORPH_OPEN, kernel)
        img_maskA = cv2.morphologyEx(img_maskA, cv2.MORPH_CLOSE, kernel)
        
        # 그결과를 원본화면의 마스크로 써서 결과값을 얻어낸 뒤 img result에 저장
        
        # 기하학적 변환을 위한 컨투어 작업
        img_result = cv2.bitwise_and(img_draw, img_draw, mask=img_maskA)
        img_result=cv2.GaussianBlur(img_result,(3,3),0)
        edgeed=cv2.Canny(img_result,75,200)
        (cnts,_)=cv2.findContours(edgeed.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        #가장 큰 부분만 찾아내기 위함.
        cnts=sorted(cnts,key=cv2.contourArea,reverse=True)[:5]
        maxc = -1
        maxc_index=-1
        for c in cnts:
            areac=cv2.contourArea(c)
            if areac<1200:
                continue
            if areac > maxc:
                maxc=areac
                maxc_index=c
           
        #가장 큰 부분을 찾아냈을때에 점들찍기
        if maxc != -1:
            cv2.drawContours(img_result,[maxc_index],0,(0,255,0),2)
            peri = cv2.arcLength(maxc_index,True)
            vertices =cv2.approxPolyDP(maxc_index,0.03 *peri,True)
            pts = vertices.reshape(-1,2)
            for x,y in pts:
                cv2.circle(img_result,(x,y),10,(0,255,0),-1)
            sm= pts.sum(axis=1)
            diff = np.diff(pts,axis=1)
            
        # 점 네개의 위치 지정
            topleft=pts[np.argmin(sm)]
            bottomleft=pts[np.argmax(sm)]
            topright=pts[np.argmin(diff)]
            bottomright=pts[np.argmax(diff)]
            
            ptsc=np.float32([topleft,topright,bottomright,bottomleft])
            #for x,y in ptsc:
                #cv2.circle(img_result,(x,y),10,(255,0,0),-1)
            w1=abs(bottomright[0]-bottomleft[0])
            w2=abs(topright[0]-topleft[0])
            h1=abs(topright[1]-bottomright[1])
            h2=abs(topleft[1]-bottomleft[1])
            if w1-w2 > 0:
                widthc=w1
            else:
                widthc=w2
            if h1-h2>0:
                heightc=h1
            else:
                heightc=h2
            # 기하학적 변환을 위한 점의 이동위치
            ptsc2=np.float32([topleft,[topleft[0]+widthc-1,topleft[1]],[topleft[0],topleft[1]+heightc-1],[topleft[0]+widthc-1,heightc-1+topleft[1]]])
            #for x,y in ptsc2:
                #cv2.circle(img_result,(x,y),10,(0,0,255),-1)

            mtrxc =cv2.getPerspectiveTransform(ptsc,ptsc2)

            resultc=cv2.warpPerspective(img_result,mtrxc,(width,height))
            resultc=cv2.cvtColor(resultc,cv2.COLOR_BGR2GRAY)
            _,resultc=cv2.threshold(resultc,1,255,cv2.THRESH_BINARY)
            img_maskA=resultc
            cv2.imshow("rect",resultc)

            
        # 라벨링
        nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(img_maskA)

        #가장 큰 부분 한군대만 찾기위함
        max = -1
        max_index = -1 

        #모든 영역의 크기를 검사해봄
        for i in range(nlabels):
            
            if i < 1:
                continue 
            
            # i번째 영역의 넓이를 얻어옴
            area = stats[i, cv2.CC_STAT_AREA]
            
            # 가장큰 영역의 index 번호를 알아옴
            if area > max:
                max = area
                max_index = i


        if max_index != -1:

            # 해당 index의 영역의 크기의 정보들을 이용
            x,y,width,height,area = stats[max_index]
            centerX,centerY = int(centroids[max_index][0]), int(centroids[max_index][1])

            # 해당영역의 값이 최소 1200이 넘을때 > 영역의 중심에는 원을 그리고, 태두리 부분에 사각형을 그려줌
            if area > 1200:
                cv2.circle(img_draw, (centerX, centerY), 7, (0,0,0), -1)
                cv2.rectangle(img_draw, (x,y), (x+width,y+height), (0,0,0),2)
            
            # 그림그리기가 true라면 영역의 중앙값의 좌표와 , 얻어낸 색상의 값을 list에 넘겨줌
            if isDraw:
                list_draw_location.append((centerX, centerY,roi_color))
        
            # 그림그리기가 false 상태면 현재 좌표들과 색상의 리스트를 history에 넘겨줌 이후 기존 리스트 값은 지워줌
            else:
                history_draw_locations.append(list_draw_location.copy())
                list_draw_location.clear()
        
        
        cv2.imshow('img_result', img_result)
        
        


    img_draw = draw(img_draw, list_draw_location)

    for draw_locations in history_draw_locations:
        img_draw = draw(img_draw, draw_locations)
    
    #마지막 으로 스티커를 붙일지 말지를 확인후 붙인다면, 원본의 배경과 배경이 없는 스티커 이미지를 합쳐줌
    if is_result and is_stikerdraw:
        h2,w2=stiker.shape[:2]
        x=point1
        y=point2
        if (y-(h2//2)>0) and (y+(h2//2)<height) and (x-(w2//2)>0) and (x+(w2//2)<width):
            roir=img_draw[y-(h2//2):y+(h2//2),x-(w2//2):x+(w2//2)]
            d1=(h2//2)*2
            d2=(w2//2)*2
            stiker_gray=cv2.cvtColor(stiker,cv2.COLOR_BGR2GRAY)
            _,stiker_mask=cv2.threshold(stiker_gray,1,255,cv2.THRESH_BINARY)
            stiker_mask_inv=cv2.bitwise_not(stiker_mask)
            stiker_mask1=cv2.bitwise_and(roir,roir,mask=stiker_mask_inv[:d1,:d2])
            added=stiker_mask1+stiker[:d1,:d2]
            img_draw[y-(h2//2):y+(h2//2),x-(w2//2):x+(w2//2)]=added
    cv2.imshow('img_draw', img_draw)
    
    
    
    
    # 메인 원본 이미지에서의 키입력 
    # `esc는 종료, 스페이스 바는 영상의 중앙 사각형으로 색을 추출하여 h의 평균값을 이용 mask 이미지에 쓰일 값을 얻음
    # r 은 스페이스바로 얻은 색으로 그림을 그림
    # x 는 화면을 모두 지워줌
    # m 은 스티커를 만드는 화면으로 넘겨줌
    # s 는 완성된 스티커를 붙여줌
    key = cv2.waitKey(1)
    if key == 27: # esc
        break
    elif key == 32: # space
        if step == 0:
            roi = img_draw2[cy-20:cy+20, cx-20:cx+20]
            roi = cv2.medianBlur(roi, 3)
            cv2.imshow("roi", roi)
            roi_c = roi.copy()
            b,g,r = cv2.split(roi_c)
            roi_color=(b.mean(),g.mean(),r.mean())
            hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
            h,s,v = cv2.split(hsv)
            color1 = h.mean()
            set_color=True
            step = 1
        elif step ==1:
            set_color=False
            step = 0
    elif key == ord('r'):
            isDraw = not isDraw
    elif key == ord('x'):
            list_draw_location.clear()
            history_draw_locations.clear()
    elif key == ord('m'):
        is_stiker=True
        stiker=makestiker()
    elif key == ord('s'):
        if is_stiker:  
            is_result=True

            
cap.release()
cv2.destroyAllWindows()


TypeError: makestiker() takes 0 positional arguments but 1 was given

TypeError: makestiker() takes 0 positional arguments but 1 was given

TypeError: makestiker() takes 0 positional arguments but 1 was given

TypeError: makestiker() takes 0 positional arguments but 1 was given

TypeError: makestiker() takes 0 positional arguments but 1 was given

...

TypeError: makestiker() takes 0 positional arguments but 1 was given

TypeError: makestiker() takes 0 positional arguments but 1 was given

TypeError: makestiker() takes 0 positional arguments but 1 was given

TypeError: makestiker() takes 0 positional arguments but 1 was given

TypeError: makestiker() takes 0 positional arguments but 1 was given